### params setting

In [142]:
import os
import random
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yaml

import torch
from qucumber.nn_states import DensityMatrix
from qucumber.nn_states import ComplexWaveFunction
from qucumber.callbacks import MetricEvaluator
import qucumber.utils.unitaries as unitaries
import qucumber.utils.training_statistics as ts
import qucumber.utils.cplx as cplx
import qucumber.utils.data as data
from qucumber.observables import ObservableBase, to_pm1
from qucumber.observables.pauli import flip_spin
import qucumber

from qulacs.gate import Pauli

with open('./params_setting_template.yaml', 'r') as yml:
    params = yaml.safe_load(yml)
    
# quantum circuit parameter
n_qubit = params["circuit_info"]["n_qubit"]
n_data = params["circuit_info"]["n_data"]
each_n_shot = int(n_data / 3**n_qubit)
state_name = params["circuit_info"]["state_name"]
noise_model = params["circuit_info"]["noise_model"]
error_rate = params["circuit_info"]["error_rate"]
# RBM architecture parameter
n_visible_unit = params["architecture_info"]["n_visible_unit"]
n_hidden_unit = params["architecture_info"]["n_hidden_unit"] 
n_aux_unit = params["architecture_info"]["n_aux_unit"]
# train parameter
lr = params["train_info"]["lr"]
pbs = params["train_info"]["positive_batch_size"]
nbs = params["train_info"]["negative_batch_size"]
n_gibbs_step = params["train_info"]["n_gibbs_step"]
period = 1
epochs = params["train_info"]["n_epoch"]
lr_drop_epoch = params["train_info"]["lr_drop_epoch"]
lr_drop_factor = params["train_info"]["lr_drop_factor"]
seed = params["train_info"]["seed"]
# sampling parameter
n_sampling = params["sampling_info"]["n_sample"]
n_copy = params["sampling_info"]["n_copy"]
# data path info
data_path = f"./data/{noise_model}/{100*error_rate}%/"

# settings
## warnings
warnings.simplefilter('ignore')

## seaborn layout
sns.set()
sns.set_style("white")

## seed
def seed_settings(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    qucumber.set_random_seed(seed, cpu=True, gpu=False)

seed_settings(seed=seed)

In [140]:
print(data_path)

./data/depolarizing/1.0%/


### quantum_gate.py

In [62]:
# 1-qubit gate
## pauli X
def X(n_qubit, target_qubit_idx):
    I = np.eye(2)
    local_X = np.array([[0,1],[1,0]])
    if target_qubit_idx==0:
        mat = local_X
    else:
        mat = I
    for i in range(n_qubit-1):
        if i+1==target_qubit_idx:
            mat = np.kron(mat, local_X)
        else:
            mat = np.kron(mat, I)
            
    return mat

## pauli Y
def Y(n_qubit, target_qubit_idx):
    I = np.eye(2)
    local_Y = np.array([[0,-1j], [1j,0]])
    if target_qubit_idx==0:
        mat = local_Y
    else:
        mat = I
    for i in range(n_qubit-1):
        if i+1==target_qubit_idx:
            mat = np.kron(mat, local_Y)
        else:
            mat = np.kron(mat, I)
            
    return mat

## pauli Z
def Z(n_qubit, target_qubit_idx):
    I = np.eye(2)
    local_Z = np.array([[1,0], [0,-1]])
    if target_qubit_idx==0:
        mat = local_Z
    else:
        mat = I
    for i in range(n_qubit-1):
        if i+1==target_qubit_idx:
            mat = np.kron(mat, local_Z)
        else:
            mat = np.kron(mat, I)
            
    return mat

## Hadamard gate
def H(n_qubit, target_qubit_idx):
    I = np.eye(2)
    local_H = np.array([[1,1], [1,-1]]) / np.sqrt(2)
    if target_qubit_idx==0:
        mat = local_H
    else:
        mat = I
    for i in range(n_qubit-1):
        if i+1==target_qubit_idx:
            mat = np.kron(mat, local_H)
        else:
            mat = np.kron(mat, I)
            
    return mat

## S gate
def S(n_qubit, target_qubit_idx):
    I = np.eye(2)
    local_S = np.array([[1,0], [0,1j]])
    if target_qubit_idx==0:
        mat = local_S
    else:
        mat = I
    for i in range(n_qubit-1):
        if i+1==target_qubit_idx:
            mat = np.kron(mat, local_S)
        else:
            mat = np.kron(mat, I)
            
    return mat

## T gate
def T(n_qubit, target_qubit_idx):
    I = np.eye(2)
    local_T = np.array([[1,0], [0,-np.exp(1j*np.pi/4)]])
    if target_qubit_idx==0:
        mat = local_T
    else:
        mat = I
    for i in range(n_qubit-1):
        if i+1==target_qubit_idx:
            mat = np.kron(mat, local_T)
        else:
            mat = np.kron(mat, I)
            
    return mat

## Rx gate
def Rx(n_qubit, target_qubit_idx, theta):
    I = np.eye(2)
    local_Rx = np.array([[np.cos(theta/2),-1j*np.sin(theta/2)], [-1j*np.sin(theta/2),np.cos(theta/2)]])
    if target_qubit_idx==0:
        mat = local_Rx
    else:
        mat = I
    for i in range(n_qubit-1):
        if i+1==target_qubit_idx:
            mat = np.kron(mat, local_Rx)
        else:
            mat = np.kron(mat, I)
            
    return mat

## Ry gate
def Ry(n_qubit, target_qubit_idx, theta):
    I = np.eye(2)
    local_Ry = np.array([[np.cos(theta/2),-np.sin(theta/2)], [-np.sin(theta/2),np.cos(theta/2)]])
    if target_qubit_idx==0:
        mat = local_Ry
    else:
        mat = I
    for i in range(n_qubit-1):
        if i+1==target_qubit_idx:
            mat = np.kron(mat, local_Ry)
        else:
            mat = np.kron(mat, I)
            
    return mat

## Rz gate
def Rz(n_qubit, target_qubit_idx, theta):
    I = np.eye(2)
    local_Rz = np.array([[np.exp(-1j*theta/2),0], [0,np.cos(1j*theta/2)]])
    if target_qubit_idx==0:
        mat = local_Rz
    else:
        mat = I
    for i in range(n_qubit-1):
        if i+1==target_qubit_idx:
            mat = np.kron(mat, local_Rz)
        else:
            mat = np.kron(mat, I)
            
    return mat

# 2-qubit gate
## CX gate
def CX(n_qubit, control_qubit_idx, target_qubit_idx):
    I = np.eye(2)
    ket_0 = np.array([[1],[0]]) 
    ket_1 = np.array([[0],[1]])
    
    mat_00 = ket_0 @ ket_0.T.conjugate() ### |0><0|
    mat_11 = ket_1 @ ket_1.T.conjugate() ### |1><1|
    
    eye_tensor = I
    
    for i in range(n_qubit-2):
        eye_tensor = np.kron(eye_tensor, I)
    
    if control_qubit_idx < target_qubit_idx:
        ### |0><0|　\otimes I \otimes ... \otimes I
        cx_mat_term1 = np.kron(mat_00, eye_tensor)
        ### |1><1| \otimes I ... \otimes X ...
        cx_mat_term2 = np.kron(mat_11, X(n_qubit-1, target_qubit_idx-1))
        
    if control_qubit_idx > target_qubit_idx:
        ### I \otimes ... \otimes I \otimes |0><0|
        cx_mat_term1 = np.kron(eye_tensor, mat_00)
        ### ... \otimes X ... \otimes |1><1|
        cx_mat_term2 = np.kron(X(n_qubit-1, target_qubit_idx), mat_11)
    
    mat = cx_mat_term1 + cx_mat_term2
    
    return mat

## CY gate
def CY(n_qubit, control_qubit_idx, target_qubit_idx):
    I = np.eye(2)
    ket_0 = np.array([[1],[0]]) 
    ket_1 = np.array([[0],[1]])
    
    mat_00 = ket_0 @ ket_0.T.conjugate() ### |0><0|
    mat_11 = ket_1 @ ket_1.T.conjugate() ### |1><1|
    
    eye_tensor = I
    
    for i in range(n_qubit-2):
        eye_tensor = np.kron(eye_tensor, I)
    
    if control_qubit_idx < target_qubit_idx:
        ### |0><0|　\otimes I \otimes ... \otimes I
        cx_mat_term1 = np.kron(mat_00, eye_tensor)
        ### |1><1| \otimes ... \otimes Y ...
        cx_mat_term2 = np.kron(mat_11, Y(n_qubit-1, target_qubit_idx-1))
        
    if control_qubit_idx > target_qubit_idx:
        ### I \otimes ... \otimes I \otimes |0><0|
        cx_mat_term1 = np.kron(eye_tensor, mat_00)
        ### ... \otimes Y ... \otimes |1><1|
        cx_mat_term2 = np.kron(Y(n_qubit-1, target_qubit_idx), mat_11)
    
    mat = cx_mat_term1 + cx_mat_term2
    
    return mat

## CZ gate
def CZ(n_qubit, control_qubit_idx, target_qubit_idx):
    I = np.eye(2)
    ket_0 = np.array([[1],[0]]) 
    ket_1 = np.array([[0],[1]])
    
    mat_00 = ket_0 @ ket_0.T.conjugate() ### |0><0|
    mat_11 = ket_1 @ ket_1.T.conjugate() ### |1><1|
    
    eye_tensor = I
    
    for i in range(n_qubit-2):
        eye_tensor = np.kron(eye_tensor, I)
    
    if control_qubit_idx < target_qubit_idx:
        ### |0><0|　\otimes I \otimes ... \otimes I
        cx_mat_term1 = np.kron(mat_00, eye_tensor)
        ### |1><1| \otimes I ... \otimes Z ...
        cx_mat_term2 = np.kron(mat_11, Z(n_qubit-1, target_qubit_idx-1))
        
    if control_qubit_idx > target_qubit_idx:
        ### I \otimes ... \otimes I \otimes |0><0|
        cx_mat_term1 = np.kron(eye_tensor, mat_00)
        ### ... \otimes Z ... \otimes |1><1|
        cx_mat_term2 = np.kron(Z(n_qubit-1, target_qubit_idx), mat_11)
    
    mat = cx_mat_term1 + cx_mat_term2
    
    return mat

## CH gate
def CH(n_qubit, control_qubit_idx, target_qubit_idx):
    I = np.eye(2)
    ket_0 = np.array([[1],[0]]) 
    ket_1 = np.array([[0],[1]])
    
    mat_00 = ket_0 @ ket_0.T.conjugate() ### |0><0|
    mat_11 = ket_1 @ ket_1.T.conjugate() ### |1><1|
    
    eye_tensor = I
    
    for i in range(n_qubit-2):
        eye_tensor = np.kron(eye_tensor, I)
    
    if control_qubit_idx < target_qubit_idx:
        ### |0><0|　\otimes I \otimes ... \otimes I
        cx_mat_term1 = np.kron(mat_00, eye_tensor)
        ### |1><1| \otimes I ... \otimes H ...
        cx_mat_term2 = np.kron(mat_11, H(n_qubit-1, target_qubit_idx-1))
        
    if control_qubit_idx > target_qubit_idx:
        ### I \otimes ... \otimes I \otimes |0><0|
        cx_mat_term1 = np.kron(eye_tensor, mat_00)
        ### ... \otimes H ... \otimes |1><1|
        cx_mat_term2 = np.kron(H(n_qubit-1, target_qubit_idx), mat_11)
    
    mat = cx_mat_term1 + cx_mat_term2
    
    return mat

## CRx gate
def CRx(n_qubit, control_qubit_idx, target_qubit_idx, theta):
    I = np.eye(2)
    ket_0 = np.array([[1],[0]]) 
    ket_1 = np.array([[0],[1]])
    
    mat_00 = ket_0 @ ket_0.T.conjugate() ### |0><0|
    mat_11 = ket_1 @ ket_1.T.conjugate() ### |1><1|
    
    eye_tensor = I
    
    for i in range(n_qubit-2):
        eye_tensor = np.kron(eye_tensor, I)
    
    if control_qubit_idx < target_qubit_idx:
        ### |0><0|　\otimes I \otimes ... \otimes I
        cx_mat_term1 = np.kron(mat_00, eye_tensor)
        ### |1><1| \otimes I ... \otimes Rx ...
        cx_mat_term2 = np.kron(mat_11, Rx(n_qubit-1, target_qubit_idx-1, theta))
        
    if control_qubit_idx > target_qubit_idx:
        ### I \otimes ... \otimes I \otimes |0><0|
        cx_mat_term1 = np.kron(eye_tensor, mat_00)
        ### ... \otimes Rx ... \otimes |1><1|
        cx_mat_term2 = np.kron(Rx(n_qubit-1, target_qubit_idx, theta), mat_11)
    
    mat = cx_mat_term1 + cx_mat_term2
    
    return mat

## CRy gate
def CRy(n_qubit, control_qubit_idx, target_qubit_idx, theta):
    I = np.eye(2)
    ket_0 = np.array([[1],[0]]) 
    ket_1 = np.array([[0],[1]])
    
    mat_00 = ket_0 @ ket_0.T.conjugate() ### |0><0|
    mat_11 = ket_1 @ ket_1.T.conjugate() ### |1><1|
    
    eye_tensor = I
    
    for i in range(n_qubit-2):
        eye_tensor = np.kron(eye_tensor, I)
    
    if control_qubit_idx < target_qubit_idx:
        ### |0><0|　\otimes I \otimes ... \otimes I
        cx_mat_term1 = np.kron(mat_00, eye_tensor)
        ### |1><1| \otimes I ... \otimes Ry ...
        cx_mat_term2 = np.kron(mat_11, Ry(n_qubit-1, target_qubit_idx-1, theta))
        
    if control_qubit_idx > target_qubit_idx:
        ### I \otimes ... \otimes I \otimes |0><0|
        cx_mat_term1 = np.kron(eye_tensor, mat_00)
        ### ... \otimes Ry ... \otimes |1><1|
        cx_mat_term2 = np.kron(Ry(n_qubit-1, target_qubit_idx, theta), mat_11)
    
    mat = cx_mat_term1 + cx_mat_term2
    
    return mat

## CRz gate
def CRz(n_qubit, control_qubit_idx, target_qubit_idx, theta):
    I = np.eye(2)
    ket_0 = np.array([[1],[0]]) 
    ket_1 = np.array([[0],[1]])
    
    mat_00 = ket_0 @ ket_0.T.conjugate() ### |0><0|
    mat_11 = ket_1 @ ket_1.T.conjugate() ### |1><1|
    
    eye_tensor = I
    
    for i in range(n_qubit-2):
        eye_tensor = np.kron(eye_tensor, I)
    
    if control_qubit_idx < target_qubit_idx:
        ### |0><0|　\otimes I \otimes ... \otimes I
        cx_mat_term1 = np.kron(mat_00, eye_tensor)
        ### |1><1| \otimes I ... \otimes Rz ...
        cx_mat_term2 = np.kron(mat_11, Rz(n_qubit-1, target_qubit_idx-1, theta))
        
    if control_qubit_idx > target_qubit_idx:
        ### I \otimes ... \otimes I \otimes |0><0|
        cx_mat_term1 = np.kron(eye_tensor, mat_00)
        ### ... \otimes Rz ... \otimes |1><1|
        cx_mat_term2 = np.kron(Rz(n_qubit-1, target_qubit_idx, theta), mat_11)
    
    mat = cx_mat_term1 + cx_mat_term2
    
    return mat

## SWAP gate
def SWAP(n_qubit, qubit_idx_1, qubit_idx_2):
    mat = CX(n_qubit, qubit_idx_1, qubit_idx_2) @ CX(n_qubit, qubit_idx_2, qubit_idx_1)
    mat = mat @ CX(n_qubit, qubit_idx_1, qubit_idx_2)
    
    return mat

# 3-qubit gate
## toffoli gate
"""
def toffoli(n_qubit, control_qubit_idx_1, control_qubit_idx_2, target_qubit_idx):
"""

'\ndef toffoli(n_qubit, control_qubit_idx_1, control_qubit_idx_2, target_qubit_idx):\n'

### error_model.py

In [7]:
def depolarizing(state, n_qubit, error_rate, target_qubit_idx):
    I = np.eye(2)
    coff_I = (1-error_rate)*I
    
    if target_qubit_idx==0:
        mat = coff_I
    else:
        mat = I
    for i in range(n_qubit-1):
        if i+1==target_qubit_idx:
            mat = np.kron(mat, coff_I)
        else:
            mat = np.kron(mat, I)
            
    depolarizing_term1 = mat @ state
    depolarizing_term2 = X(n_qubit, target_qubit_idx)@state@X(n_qubit, target_qubit_idx) + Y(n_qubit, target_qubit_idx)@state@Y(n_qubit, target_qubit_idx) + Z(n_qubit, target_qubit_idx)@state@Z(n_qubit, target_qubit_idx)
    
    return depolarizing_term1 + (error_rate/3)*depolarizing_term2
    
def unitary(state, n_qubit, theta, target_qubit_idx):
    
    return Rx(n_qubit, target_qubit_idx, theta) @ state @ Rx(n_qubit, target_qubit_idx, theta).T.conjugate()

### target_circuit.py

In [126]:
def init_state(n_qubit, state_name):
    ket_0 = np.array([[1],[0]]) 
    init_state = ket_0
    
    for i in range(2**(n_qubit-1)-1):
        init_state = np.append(init_state, np.array([[0],[0]]), axis=0) # |00...0>
    
    if state_name == "density_matrix":
        init_state_vec = init_state
        init_state = init_state_vec @ init_state_vec.T.conjugate() # |00...0><00...0|
    
    return init_state

def Bell(n_qubit, state_name, error_model, error_rate):
    if state_name == "state_vector":
        if error_model == "ideal":
            state = init_state(n_qubit, state_name)
            state = H(n_qubit,0) @ state
            state = CX(n_qubit,0,1) @ state
    
    if state_name == "density_matrix":
        if error_model == "ideal":
            state = init_state(n_qubit, state_name)
            state = H(n_qubit,0) @ state @ H(n_qubit,0).T.conjugate()
            state = CX(n_qubit,0,1) @ state @ CX(n_qubit,0,1).T.conjugate()
        
        if error_model == "depolarizing":
            state = init_state(n_qubit, state_name)
            state = H(n_qubit,0) @ state @ H(n_qubit,0).T.conjugate()
            state = depolarizing(state, n_qubit, error_rate, 0)
            state = CX(n_qubit,0,1) @ state @ CX(n_qubit,0,1).T.conjugate()
            state = depolarizing(state, n_qubit, error_rate, 0)
            state = depolarizing(state, n_qubit, error_rate, 1)
        
        if error_model == "unitary":
            state = init_state(n_qubit, state_name)
            state = H(n_qubit,0) @ state @ H(n_qubit,0).T.conjugate()
            state = unitary(state, n_qubit, np.sqrt(error_rate), 0)
            state = CX(n_qubit,0,1) @ state @ CX(n_qubit,0,1).T.conjugate()
            state = unitary(state, n_qubit, np.sqrt(error_rate), 0)
            state = unitary(state, n_qubit, np.sqrt(error_rate), 1)
            
        if error_model == "depolarizing&unitary":
            state = init_state(n_qubit, state_name)
            state = H(n_qubit,0) @ state @ H(n_qubit,0).T.conjugate()
            state = depolarizing(state, n_qubit, error_rate, 0)
            state = unitary(state, n_qubit, np.sqrt(error_rate), 0)
            state = CX(n_qubit,0,1) @ state @ CX(n_qubit,0,1).T.conjugate()
            state = depolarizing(state, n_qubit, error_rate, 0)
            state = depolarizing(state, n_qubit, error_rate, 1)
            state = unitary(state, n_qubit, np.sqrt(error_rate), 0)
            state = unitary(state, n_qubit, np.sqrt(error_rate), 1)
            
    return state

def GHZ(n_qubit, state_name, error_model, error_rate):
    if state_name == "state_vector":
        if error_model == "ideal":
            state = init_state(n_qubit, state_name)
            state = H(n_qubit, 0) @ state
            for i in range(n_qubit-1):
                state = CX(n_qubit, 0, i+1) @ state
        
    if state_name == "density_matrix":
        if error_model == "ideal":
            state = init_state(n_qubit, state_name)
            state = H(n_qubit,0) @ state @ H(n_qubit,0).T.conjugate()
        
            for i in range(n_qubit-1):
                state = CX(n_qubit,0,i+1) @ state @ CX(n_qubit,0,i+1).T.conjugate()
        
        if error_model == "depolarizing":
            state = init_state(n_qubit, state_name)
            state = H(n_qubit,0) @ state @ H(n_qubit,0).T.conjugate()
            state = depolarizing(state, n_qubit, error_rate, 0)
            
            for i in range(n_qubit-1):
                state = CX(n_qubit,0,i+1) @ state @ CX(n_qubit,0,i+1).T.conjugate()
                state = depolarizing(state, n_qubit, error_rate, 0)
                state = depolarizing(state, n_qubit, error_rate, i+1)
        
        if error_model == "unitary":
            state = init_state(n_qubit, state_name)
            state = H(n_qubit,0) @ state @ H(n_qubit,0).T.conjugate()
            state = unitary(state, n_qubit, np.sqrt(error_rate), 0)
            
            for i in range(n_qubit-1):
                state = CX(n_qubit,0,i+1) @ state @ CX(n_qubit,0,i+1).T.conjugate()
                state = unitary(state, n_qubit, np.sqrt(error_rate), 0)
                state = unitary(state, n_qubit, np.sqrt(error_rate), i+1)
        
        if error_model == "depolarizing&unitary":
            state = init_state(n_qubit, state_name)
            state = H(n_qubit,0) @ state @ H(n_qubit,0).T.conjugate()
            state = depolarizing(state, n_qubit, error_rate, 0)
            state = unitary(state, n_qubit, np.sqrt(error_rate), 0)
            
            for i in range(n_qubit-1):
                state = CX(n_qubit,0,i+1) @ state @ CX(n_qubit,0,i+1).T.conjugate()
                state = depolarizing(state, n_qubit, error_rate, 0)
                state = unitary(state, n_qubit, np.sqrt(error_rate), 0)
                state = depolarizing(state, n_qubit, error_rate, i+1)
                state = unitary(state, n_qubit, np.sqrt(error_rate), i+1)
            
    return state
    
"""
def random_circuit():

def ising_model()
"""

'\ndef random_circuit():\n\ndef ising_model()\n'

### projection_operator.py

In [160]:
def X_basis(n_qubit, target_idx, classical_bit):
    I = np.eye(2**n_qubit)
    P = X(n_qubit, target_idx)
    
    if bit_prob == "0":
        operator = (I+P) / 2
    if bit_prob == "1":
        operator = (I-P) / 2
    
    return operator

def Y_basis(n_qubit, target_idx, classical_bit):
    I = np.eye(2**n_qubit)
    P = Y(n_qubit, target_idx)
    
    if bit_prob == "0":
        operator = (I+P) / 2
    if bit_prob == "1":
        operator = (I-P) / 2
    
    return operator

def Z_basis(n_qubit, target_idx, classical_bit):
    I = np.eye(2**n_qubit)
    P = Z(n_qubit, target_idx)
    
    if bit_prob == "0":
        operator = (I+P) / 2
    if bit_prob == "1":
        operator = (I-P) / 2
    
    return operator

### measurement.py

In [236]:
def pauli(n_qubit, state_name, error_model, pauli_str_list):
    target_qubit_idx_list = np.arange(n_qubit)
    pauli_meas_dict = {"X":X_basis, "Y":Y_basis, "Z":Z_basis}
    measurement_label_list = []
    measurement_result_list = []
    
    rho_0 = Bell(n_qubit, state_name, error_model, error_rate)
    rho_1 = Bell(n_qubit, state_name, error_model, error_rate)
    
    for target_qubit_idx, pauli_str in zip(target_qubit_idx_list, pauli_str_list):
        print(f"target_qubit_idx : {target_qubit_idx}")
        print(f"pauli_str : {pauli_str}")
        operator0, operator1 = pauli_meas_dict[pauli_str](n_qubit, target_qubit_idx)
        
        print(f"M0 : {operator0}")
        print(f"M1 : {operator1}")
        print(f"rho0 : {rho_0}")
        print(f"rho1 : {rho_1}")
        
        p0 = np.real(np.trace(operator0 @ rho_0))
        p1 = np.real(np.trace(operator1 @ rho_1))
        
        print(f"p0 : {p0}")
        print(f"p1 : {p1}")
        
        rho_0 = (operator0@rho_0@operator0)/ np.trace(operator0@operator0@rho_0)
        rho_1 = (operator1@rho_1@operator1) / np.trace(operator1@operator1@rho_1)
        
        measurement_label_list.append(pauli_str)
        measurement_result_list.append(np.random.choice(["0","1"], p=[p0,p1]))
        
    return measurement_label_list, measurement_result_list

In [237]:
label_list, measurement_result_list = pauli(n_qubit, "density_matrix", "depolarizing", ["Z", "Z"])
label_list

target_qubit_idx : 0
pauli_str : Z
M0 : [[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
M1 : [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
rho0 : [[0.49337778+0.j 0.        +0.j 0.        +0.j 0.48026548+0.j]
 [0.        +0.j 0.00662222+0.j 0.        +0.j 0.        +0.j]
 [0.        +0.j 0.        +0.j 0.00662222+0.j 0.        +0.j]
 [0.48026548+0.j 0.        +0.j 0.        +0.j 0.49337778+0.j]]
rho1 : [[0.49337778+0.j 0.        +0.j 0.        +0.j 0.48026548+0.j]
 [0.        +0.j 0.00662222+0.j 0.        +0.j 0.        +0.j]
 [0.        +0.j 0.        +0.j 0.00662222+0.j 0.        +0.j]
 [0.48026548+0.j 0.        +0.j 0.        +0.j 0.49337778+0.j]]
p0 : 0.49999999999999983
p1 : 0.49999999999999983
target_qubit_idx : 1
pauli_str : Z
M0 : [[1. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 0.]]
M1 : [[0. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 1.]]
rho0 : [[0.98675556+0.j 0.        +0.j 0.        +0.j 0.        +0.j]
 [0.        +0.j 0.0132444

ValueError: probabilities do not sum to 1

In [ ]:
def pauli(n_qubit, state_name, error_model, pauli_str_list):
    target_qubit_idx_list = np.arange(n_qubit)
    pauli_meas_dict = {"X":X_basis, "Y":Y_basis, "Z":Z_basis}
    
    measurement_label_list = []
    measurement_result_list = []
    
    rho_0 = Bell(n_qubit, state_name, error_model, error_rate)
    for target_qubit_idx, pauli_str in zip(target_qubit_idx_list, pauli_str_list):
        for
    

### dataset.py

In [113]:
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import itertools

def generate(n_qubit, state_name, n_shot, error_model):
    meas_pattern_list = []
    meas_label_list = []
    meas_result_list = []
    
    pauli_meas_label = ["X", "Y", "Z"]
    #operator_pattern_list = itertools.product(pauli_meas_label)
    
    for meas_pattern in tqdm(itertools.product(pauli_meas_label, repeat=n_qubit)):
        meas_pattern_list.append(meas_pattern)
        print(f"measurement pattern : {meas_pattern}")
        
        for i in tqdm(range(int(n_shot/3**n_qubit))):
            label, result = pauli(n_qubit, state_name, error_model, meas_pattern)
            meas_label_list.append(label)
            meas_result_list.append(result)
    
    meas_pattern_df = pd.DataFrame({"measurement_pattern":meas_pattern_list})
    meas_pattern_df["measurement_pattern"] = meas_pattern_df["measurement_pattern"].apply(lambda x: " ".join(x))
    train_df = pd.DataFrame({"measurement_label":meas_label_list, "measurement_result":meas_result_list})
    train_df["measurement_label"] = train_df["measurement_label"].apply(lambda x: " ".join(x))
    train_df["measurement_result"] = train_df["measurement_result"].apply(lambda x: " ".join(x))
    
    return meas_pattern_df, train_df

"""
def data_export(meas_pattern_df, train_df):
    meas_pattern_df.to_csv("./{}/data/measurement_pattern.txt", header=False, index=False)
    train_df.to_csv("./data//measurement_label.txt", columns = ["measurement_label"], header=False, index=False)
    train_df.to_csv("./data/{}/measurement_result.txt", columns = ["measurement_result"], header=False, index=False)
"""

'\ndef data_export(meas_pattern_df, train_df):\n    meas_pattern_df.to_csv("./{}/data/measurement_pattern.txt", header=False, index=False)\n    train_df.to_csv("./data//measurement_label.txt", columns = ["measurement_label"], header=False, index=False)\n    train_df.to_csv("./data/{}/measurement_result.txt", columns = ["measurement_result"], header=False, index=False)\n'

### debug

In [128]:
init_state(3, "density_matrix")

array([[1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0]])

In [131]:
ideal_bell_state = Bell(2, "density_matrix", "ideal", error_rate)
ideal_bell_state

array([[0.5, 0. , 0. , 0.5],
       [0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. ],
       [0.5, 0. , 0. , 0.5]])

In [133]:
np.savetxt('sample.txt', np.imag(ideal_bell_state))

In [114]:
meas_pattern_df, train_df = generate(n_qubit, n_shot)

0it [00:00, ?it/s]

measurement pattern : ('X', 'X')


  0%|          | 0/1111 [00:00<?, ?it/s]

measurement pattern : ('X', 'Y')


  0%|          | 0/1111 [00:00<?, ?it/s]

...

  0%|          | 0/1111 [00:00<?, ?it/s]

measurement pattern : ('Z', 'Y')


  0%|          | 0/1111 [00:00<?, ?it/s]

measurement pattern : ('Z', 'Z')


  0%|          | 0/1111 [00:00<?, ?it/s]

In [115]:
train_df

,measurement_label,measurement_result
0,X X,0 1
1,X X,1 0
2,X X,1 1
3,X X,1 0
4,X X,1 0
...,...,...
9994,Z Z,1 0
9995,Z Z,0 0
9996,Z Z,0 0
9997,Z Z,0 1


In [116]:
import torch

In [117]:
import optuna

ModuleNotFoundError: No module named 'optuna'

In [118]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 KB 3.8 MB/s eta 0:00:0000:0100:01


In [134]:
3**10

59049

In [144]:
ideal_state_vector = Bell(n_qubit, "state_vector", "ideal", error_rate)
np.real(ideal_state_vector)

array([[0.70710678],
       [0.        ],
       [0.        ],
       [0.70710678]])

In [145]:
np.imag(ideal_state_vector)

array([[0.],
       [0.],
       [0.],
       [0.]])

In [148]:
df = pd.DataFrame({"Re":np.real(ideal_state_vector.reshape(-1)), "Im":np.imag(ideal_state_vector.reshape(-1))})
df

,Re,Im
0,0.707107,0.0
1,0.000000,0.0
2,0.000000,0.0
3,0.707107,0.0


In [211]:
ideal_rho = Bell(n_qubit, "density_matrix", "depolarizing", 0.01)
ideal_rho

array([[0.49337778+0.j, 0.        +0.j, 0.        +0.j, 0.48026548+0.j],
       [0.        +0.j, 0.00662222+0.j, 0.        +0.j, 0.        +0.j],
       [0.        +0.j, 0.        +0.j, 0.00662222+0.j, 0.        +0.j],
       [0.48026548+0.j, 0.        +0.j, 0.        +0.j, 0.49337778+0.j]])

In [212]:
M_0, M_1 = Z_basis(2, 0)

In [213]:
M_0

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [214]:
M_1

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

In [215]:
p_0 = np.trace(M_0@ideal_rho)
p_0

(0.49999999999999983+0j)

In [216]:
p_1 = np.trace(M_1@ideal_rho)
p_1

(0.49999999999999983+0j)

In [217]:
rho_0 = (M_0@ideal_rho@M_0) / np.trace(M_0@M_0@ideal_rho)
rho_0

array([[0.98675556+0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j],
       [0.        +0.j, 0.01324444+0.j, 0.        +0.j, 0.        +0.j],
       [0.        +0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j],
       [0.        +0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j]])

In [218]:
rho_1 = (M_1@ideal_rho@M_1) / np.trace(M_1@M_1@ideal_rho)
rho_1

array([[0.        +0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j],
       [0.        +0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j],
       [0.        +0.j, 0.        +0.j, 0.01324444+0.j, 0.        +0.j],
       [0.        +0.j, 0.        +0.j, 0.        +0.j, 0.98675556+0.j]])

In [219]:
M_0, M_1 = Z_basis(2, 1)

In [220]:
M_0

array([[1., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 0.]])

In [221]:
M_1

array([[0., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 1.]])

In [222]:
p_00 = np.trace(rho_0@M_0)
p_00

(0.9867555555555557+0j)

In [223]:
p_01 = np.trace(rho_0@M_1)
p_01

(0.013244444444444446+0j)

In [224]:
p_10 = np.trace(rho_1@M_0)
p_10

(0.013244444444444446+0j)

In [225]:
p_11 = np.trace(rho_1@M_1)
p_11

(0.9867555555555557+0j)